# Tensorflow Hub Model with Five Percent Data Augmenetation (Top 10 Diagnosis)

In [1]:
import pickle, random
import numpy as np

In [2]:
import pandas as pd

In [3]:
def get_sorted_dict(D):
    return {k: v for k, v in sorted(D.items(), key=lambda item: item[1], reverse=1)}

In [4]:
winogender_sentences = "all_sentences.tsv"

In [5]:
winogender_occupations = "occupations-stats.tsv"

In [6]:
import csv

wino_occupations = []

In [7]:
with open(winogender_occupations, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        wino_occupations.append(lines["occupation"])

In [8]:
# wino_occupations

In [9]:
len(wino_occupations)

60

In [10]:
len(set(wino_occupations))

60

### Load Text Classification Hub Model

In [11]:
import tensorflow as tf

In [12]:
# my_model directory
# !ls ../saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls ../saved_model/one_quarter_percent_models/

my_model_txt_classifier_hub_with_one_quarter_percent_extra_data_rand_1
my_model_txt_classifier_hub_with_one_quarter_percent_extra_data_rand_10
my_model_txt_classifier_hub_with_one_quarter_percent_extra_data_rand_2
my_model_txt_classifier_hub_with_one_quarter_percent_extra_data_rand_3
my_model_txt_classifier_hub_with_one_quarter_percent_extra_data_rand_4
my_model_txt_classifier_hub_with_one_quarter_percent_extra_data_rand_5
my_model_txt_classifier_hub_with_one_quarter_percent_extra_data_rand_6
my_model_txt_classifier_hub_with_one_quarter_percent_extra_data_rand_7
my_model_txt_classifier_hub_with_one_quarter_percent_extra_data_rand_8
my_model_txt_classifier_hub_with_one_quarter_percent_extra_data_rand_9


In [13]:
!ls ../saved_model/one_quarter_percent_models/my_model_txt_classifier_hub_with_one_quarter_percent_extra_data_rand_6

assets         saved_model.pb variables


In [14]:
hub_model = tf.keras.models.load_model('../saved_model/one_quarter_percent_models/my_model_txt_classifier_hub_with_one_quarter_percent_extra_data_rand_6/')
# Check its architecture
hub_model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_5 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_10 (Dense)             (None, 16)                336       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


## Test on Repaired Input Tokens

### A. Direct gender noun comparisons (e.g. My boyfriend/My girlfriend)

In [15]:
mode = "direct-gender-noun"

In [16]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [17]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [18]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [19]:
noun_error = {**noun_error1, **noun_error2}

In [20]:
# noun_error 

In [21]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [22]:
noun_error1

['my aunt',
 'my daughter',
 'my mother',
 'my wife',
 'this girl',
 'my sister',
 'this woman',
 'my girlfriend']

In [23]:
noun_error2

['my uncle',
 'my son',
 'my father',
 'my brother',
 'my husband',
 'this boy',
 'this man',
 'my boyfriend']

In [24]:
noun_error = get_sorted_dict(noun_error)

In [25]:
# TODO : sort the "my X", "this X" "He", "She"

In [26]:
# noun_error

In [27]:
# noun_error = list(noun_error)[:10]

In [28]:
# noun_error = list(noun_error)[:5]

In [29]:
noun_error = list(noun_error)

In [30]:
# noun_error

In [31]:
input1, input2, input3 = [],[], []

In [32]:
input_pairs, input_truple = [], []

In [33]:
occupation_list = []

In [34]:
i = 0
tmp, tmp2 ="", ""

In [35]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [36]:
f_input, m_input, neut_input = "", "", ""

In [37]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
#             print("male_sentence: ", lines["sentence"])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
#             print("female_sentence: ", lines["sentence"])
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" #lines["sentid"]
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
#             print("female_sentence: ", lines["sentence"])
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" #lines["sentid"]
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1
#         if i >= 6:
#             break

In [38]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [39]:
# for i in range(len(list(noun_error.keys()))):
#     print(list(noun_error.keys())[i].lstrip("the").strip())

In [40]:
# random.choice()

In [41]:
fairness_errors = []
all_inputs = []
count = 0

In [42]:
# TODO:
#     * check for mispredictions
#     * save tested inputs, input pairs, inputs leading to fairness errors and prediction errors
#     * do for all test sets (a to f) i.e. other bias tests : name, race e.t.c

In [43]:
# TODO "the" vs. "The" in str.replace

In [44]:

for i in range(len(input_truple)):
#     print(input_truple[i])
#     print(input_truple[i][0])
#     print(input_truple[i][1])
#     print(occupation_list[i])
    if occupation_list[i] in wino_occupations:
#         for k in range(len(noun_error1))
#         for j in range(len(list(noun_error.keys()))):
#         for j in range(max([len(noun_error1), len(noun_error2)])):
        for j in range(len(noun_error1)) :
            count += 1
            input1, input2 = "", ""
#             input1 = input_truple[i][0].replace(occupation_list[i], list(noun_error.keys())[j].lstrip("the").strip())
#             input2 = input_truple[i][1].replace(occupation_list[i], list(noun_error.keys())[j].lstrip("the").strip())
            if ("the " + occupation_list[i]) in  input_truple[i][0]:
                input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))
            
        
            input1, input2 = "", ""
            if ("the " + occupation_list[i]) in  input_truple[i][1]:
                input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))   
    
    
            input1, input2 = "", ""
            if ("the " + occupation_list[i]) in  input_truple[i][2]:
                input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))   
                
            if len(all_inputs) >= 3000:
                break

    if len(all_inputs) >= 3000:
        break


In [45]:
len(all_inputs)

3000

In [46]:
len(fairness_errors)

241

### B. Random gender noun comparisons (e.g. My boyfriend/My mother)

In [47]:
mode = "random-gender-noun"

In [48]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [49]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [50]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [51]:
noun_error = {**noun_error1, **noun_error2}

In [52]:
# noun_error 

In [53]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [54]:
noun_error1

['my aunt',
 'my girlfriend',
 'my daughter',
 'my wife',
 'my mother',
 'this woman',
 'this girl',
 'my sister',
 'my mom']

In [55]:
noun_error2

['my brother',
 'my father',
 'this boy',
 'my boyfriend',
 'my uncle',
 'my husband',
 'this man',
 'my dad',
 'my son']

In [56]:
noun_error = get_sorted_dict(noun_error)

In [57]:
# TODO : sort the "my X", "this X" "He", "She"

In [58]:
# noun_error

In [59]:
# noun_error = list(noun_error)[:10]

In [60]:
# noun_error = list(noun_error)[:5]

In [61]:
noun_error = list(noun_error)

In [62]:
# noun_error

In [63]:
input1, input2, input3 = [],[], []

In [64]:
input_pairs, input_truple = [], []

In [65]:
occupation_list = []

In [66]:
i = 0
tmp, tmp2 ="", ""

In [67]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [68]:
f_input, m_input, neut_input = "", "", ""

In [69]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
#             print("male_sentence: ", lines["sentence"])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
#             print("female_sentence: ", lines["sentence"])
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" #lines["sentid"]
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
#             print("female_sentence: ", lines["sentence"])
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" #lines["sentid"]
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1
#         if i >= 6:
#             break

In [70]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [71]:
# for i in range(len(list(noun_error.keys()))):
#     print(list(noun_error.keys())[i].lstrip("the").strip())

In [72]:
# random.choice()

In [73]:
fairness_errors = []
all_inputs = []
count = 0

In [74]:
# TODO:
#     * check for mispredictions
#     * save tested inputs, input pairs, inputs leading to fairness errors and prediction errors
#     * do for all test sets (a to f) i.e. other bias tests : name, race e.t.c

In [75]:
# TODO "the" vs. "The" in str.replace

In [76]:

for i in range(len(input_truple)):
#     print(input_truple[i])
#     print(input_truple[i][0])
#     print(input_truple[i][1])
#     print(occupation_list[i])
    if occupation_list[i] in wino_occupations:
#         for k in range(len(noun_error1))
#         for j in range(len(list(noun_error.keys()))):
#         for j in range(max([len(noun_error1), len(noun_error2)])):
        for j in range(len(noun_error1)) :
            count += 1
            input1, input2 = "", ""
#             input1 = input_truple[i][0].replace(occupation_list[i], list(noun_error.keys())[j].lstrip("the").strip())
#             input2 = input_truple[i][1].replace(occupation_list[i], list(noun_error.keys())[j].lstrip("the").strip())
            if ("the " + occupation_list[i]) in  input_truple[i][0]:
                input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))
            
        
            input1, input2 = "", ""
            if ("the " + occupation_list[i]) in  input_truple[i][1]:
                input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))   
    
    
            input1, input2 = "", ""
            if ("the " + occupation_list[i]) in  input_truple[i][2]:
                input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))   
                
            if len(all_inputs) >= 3000:
                break

    if len(all_inputs) >= 3000:
        break


In [77]:
len(all_inputs)

3000

In [78]:
len(fairness_errors)

170

### C. Test for Indirect Gender Bias, i.e. Occupational Bias

In [79]:
mode = "gender-occupation-noun"

In [80]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [81]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [82]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [83]:
noun_error = {**noun_error1, **noun_error2}

In [84]:
# noun_error 

In [85]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [86]:
noun_error1

['the writer',
 'the teacher',
 'the cleaner',
 'the auditor',
 'the counselor',
 'the nurse',
 'the the paralegal',
 'the editor',
 'the tailor',
 'the clerk']

In [87]:
noun_error2

['the cook',
 'the analyst',
 'the engineer',
 'the mechanic',
 'the technician',
 'the supervisor',
 'the salesperson',
 'the farmer',
 'the chief',
 'the lawyer']

In [88]:
noun_error = get_sorted_dict(noun_error)

In [89]:
# TODO : sort the "my X", "this X" "He", "She"

In [90]:
# noun_error

In [91]:
# noun_error = list(noun_error)[:10]

In [92]:
# noun_error = list(noun_error)[:5]

In [93]:
noun_error = list(noun_error)

In [94]:
# noun_error

In [95]:
input1, input2, input3 = [],[], []

In [96]:
input_pairs, input_truple = [], []

In [97]:
occupation_list = []

In [98]:
i = 0
tmp, tmp2 ="", ""

In [99]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [100]:
f_input, m_input, neut_input = "", "", ""

In [101]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
#             print("male_sentence: ", lines["sentence"])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
#             print("female_sentence: ", lines["sentence"])
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" #lines["sentid"]
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
#             print("female_sentence: ", lines["sentence"])
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" #lines["sentid"]
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1
#         if i >= 6:
#             break

In [102]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [103]:
# for i in range(len(list(noun_error.keys()))):
#     print(list(noun_error.keys())[i].lstrip("the").strip())

In [104]:
# random.choice()

In [105]:
fairness_errors = []
all_inputs = []
count = 0

In [106]:
# TODO:
#     * check for mispredictions
#     * save tested inputs, input pairs, inputs leading to fairness errors and prediction errors
#     * do for all test sets (a to f) i.e. other bias tests : name, race e.t.c

In [107]:
# TODO "the" vs. "The" in str.replace

In [108]:

for i in range(len(input_truple)):
#     print(input_truple[i])
#     print(input_truple[i][0])
#     print(input_truple[i][1])
#     print(occupation_list[i])
    if occupation_list[i] in wino_occupations:
#         for k in range(len(noun_error1))
#         for j in range(len(list(noun_error.keys()))):
#         for j in range(max([len(noun_error1), len(noun_error2)])):
        for j in range(len(noun_error1)) :
            count += 1
            input1, input2 = "", ""
#             input1 = input_truple[i][0].replace(occupation_list[i], list(noun_error.keys())[j].lstrip("the").strip())
#             input2 = input_truple[i][1].replace(occupation_list[i], list(noun_error.keys())[j].lstrip("the").strip())
            if ("the " + occupation_list[i]) in  input_truple[i][0]:
                input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))
            
        
            input1, input2 = "", ""
            if ("the " + occupation_list[i]) in  input_truple[i][1]:
                input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))   
    
    
            input1, input2 = "", ""
            if ("the " + occupation_list[i]) in  input_truple[i][2]:
                input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))   
                
            if len(all_inputs) >= 3000:
                break

    if len(all_inputs) >= 3000:
        break


In [109]:
len(all_inputs)

3000

In [110]:
len(fairness_errors)

310

### D. Test for   for Indirect Gender Bias, i.e. Name Bias

In [111]:
mode = "gender-name-noun"

In [112]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [113]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [114]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [115]:
noun_error = {**noun_error1, **noun_error2}

In [116]:
# noun_error 

In [117]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [118]:
noun_error1

['Deborah',
 'Sharon',
 'Kimberly',
 'Carol',
 'Michelle',
 'Karen',
 'Sandra',
 'Laura',
 'Nancy',
 'Barbara']

In [119]:
noun_error2

['Christopher',
 'Joshua',
 'Thomas',
 'James',
 'Timothy',
 'Mark',
 'Robert ',
 'George',
 'Edward',
 'Anthony']

In [120]:
noun_error = get_sorted_dict(noun_error)

In [121]:
# TODO : sort the "my X", "this X" "He", "She"

In [122]:
# noun_error

In [123]:
# noun_error = list(noun_error)[:10]

In [124]:
# noun_error = list(noun_error)[:5]

In [125]:
noun_error = list(noun_error)

In [126]:
# noun_error

In [127]:
input1, input2, input3 = [],[], []

In [128]:
input_pairs, input_truple = [], []

In [129]:
occupation_list = []

In [130]:
i = 0
tmp, tmp2 ="", ""

In [131]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [132]:
f_input, m_input, neut_input = "", "", ""

In [133]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
#             print("male_sentence: ", lines["sentence"])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
#             print("female_sentence: ", lines["sentence"])
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" #lines["sentid"]
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
#             print("female_sentence: ", lines["sentence"])
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" #lines["sentid"]
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1
#         if i >= 6:
#             break

In [134]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [135]:
# for i in range(len(list(noun_error.keys()))):
#     print(list(noun_error.keys())[i].lstrip("the").strip())

In [136]:
# random.choice()

In [137]:
fairness_errors = []
all_inputs = []
count = 0

In [138]:
# TODO:
#     * check for mispredictions
#     * save tested inputs, input pairs, inputs leading to fairness errors and prediction errors
#     * do for all test sets (a to f) i.e. other bias tests : name, race e.t.c

In [139]:
# TODO "the" vs. "The" in str.replace

In [140]:

for i in range(len(input_truple)):
#     print(input_truple[i])
#     print(input_truple[i][0])
#     print(input_truple[i][1])
#     print(occupation_list[i])
    if occupation_list[i] in wino_occupations:
#         for k in range(len(noun_error1))
#         for j in range(len(list(noun_error.keys()))):
#         for j in range(max([len(noun_error1), len(noun_error2)])):
        for j in range(len(noun_error1)) :
            count += 1
            input1, input2 = "", ""
#             input1 = input_truple[i][0].replace(occupation_list[i], list(noun_error.keys())[j].lstrip("the").strip())
#             input2 = input_truple[i][1].replace(occupation_list[i], list(noun_error.keys())[j].lstrip("the").strip())
            if ("the " + occupation_list[i]) in  input_truple[i][0]:
                input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))
            
        
            input1, input2 = "", ""
            if ("the " + occupation_list[i]) in  input_truple[i][1]:
                input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))   
    
    
            input1, input2 = "", ""
            if ("the " + occupation_list[i]) in  input_truple[i][2]:
                input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))   
                
            if len(all_inputs) >= 3000:
                break

    if len(all_inputs) >= 3000:
        break


In [141]:
len(all_inputs)

3000

In [142]:
len(fairness_errors)

441

### E. Test for   for Indirect Racial Bias, i.e. Name Bias

In [143]:
mode = "racial-name-noun"

In [144]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [145]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [146]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [147]:
noun_error = {**noun_error1, **noun_error2}

In [148]:
# noun_error 

In [149]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [150]:
noun_error1

['Josh',
 'Melanie',
 'Justin',
 'Stephanie',
 'Harry',
 'Jack',
 'Katie',
 'Nancy',
 'Amanda',
 'Ellen']

In [151]:
noun_error2

['Malik',
 'Lamar',
 'Jasmine',
 'Jamel',
 'Nichelle',
 'Latisha',
 'Darnell',
 'Leroy',
 'Alonzo',
 'Shereen']

In [152]:
noun_error = get_sorted_dict(noun_error)

In [153]:
# TODO : sort the "my X", "this X" "He", "She"

In [154]:
# noun_error

In [155]:
# noun_error = list(noun_error)[:10]

In [156]:
# noun_error = list(noun_error)[:5]

In [157]:
noun_error = list(noun_error)

In [158]:
# noun_error

In [159]:
input1, input2, input3 = [],[], []

In [160]:
input_pairs, input_truple = [], []

In [161]:
occupation_list = []

In [162]:
i = 0
tmp, tmp2 ="", ""

In [163]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [164]:
f_input, m_input, neut_input = "", "", ""

In [165]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
#             print("male_sentence: ", lines["sentence"])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
#             print("female_sentence: ", lines["sentence"])
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" #lines["sentid"]
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
#             print("female_sentence: ", lines["sentence"])
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" #lines["sentid"]
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1
#         if i >= 6:
#             break

In [166]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [167]:
# for i in range(len(list(noun_error.keys()))):
#     print(list(noun_error.keys())[i].lstrip("the").strip())

In [168]:
# random.choice()

In [169]:
fairness_errors = []
all_inputs = []
count = 0

In [170]:
# TODO:
#     * check for mispredictions
#     * save tested inputs, input pairs, inputs leading to fairness errors and prediction errors
#     * do for all test sets (a to f) i.e. other bias tests : name, race e.t.c

In [171]:
# TODO "the" vs. "The" in str.replace

In [172]:

for i in range(len(input_truple)):
#     print(input_truple[i])
#     print(input_truple[i][0])
#     print(input_truple[i][1])
#     print(occupation_list[i])
    if occupation_list[i] in wino_occupations:
#         for k in range(len(noun_error1))
#         for j in range(len(list(noun_error.keys()))):
#         for j in range(max([len(noun_error1), len(noun_error2)])):
        for j in range(len(noun_error1)) :
            count += 1
            input1, input2 = "", ""
#             input1 = input_truple[i][0].replace(occupation_list[i], list(noun_error.keys())[j].lstrip("the").strip())
#             input2 = input_truple[i][1].replace(occupation_list[i], list(noun_error.keys())[j].lstrip("the").strip())
            if ("the " + occupation_list[i]) in  input_truple[i][0]:
                input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))
            
        
            input1, input2 = "", ""
            if ("the " + occupation_list[i]) in  input_truple[i][1]:
                input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))   
    
    
            input1, input2 = "", ""
            if ("the " + occupation_list[i]) in  input_truple[i][2]:
                input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))   
                
            if len(all_inputs) >= 3000:
                break

    if len(all_inputs) >= 3000:
        break


In [173]:
len(all_inputs)

3000

In [174]:
len(fairness_errors)

338

### F. Test for Neutral (Sentiment) Sentences

In [175]:
mode = "neutral-sentiments"

In [176]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [177]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [178]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [179]:
noun_error = {**noun_error1, **noun_error2}

In [180]:
# noun_error 

In [181]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [182]:
noun_error1

['my aunt',
 'my daughter',
 'my wife',
 'my mother',
 'my girlfriend',
 'this girl',
 'this woman',
 'my sister',
 'my mom',
 'the writer']

In [183]:
noun_error2

['my son',
 'my uncle',
 'my father',
 'my husband',
 'my brother',
 'this boy',
 'this man',
 'my boyfriend',
 'my dad',
 'the cook']

In [184]:
noun_error = get_sorted_dict(noun_error)

In [185]:
# TODO : sort the "my X", "this X" "He", "She"

In [186]:
# noun_error

In [187]:
# noun_error = list(noun_error)[:10]

In [188]:
# noun_error = list(noun_error)[:5]

In [189]:
noun_error = list(noun_error)

In [190]:
# noun_error

In [191]:
input1, input2, input3 = [],[], []

In [192]:
input_pairs, input_truple = [], []

In [193]:
occupation_list = []

In [194]:
i = 0
tmp, tmp2 ="", ""

In [195]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [196]:
f_input, m_input, neut_input = "", "", ""

In [197]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
#             print("male_sentence: ", lines["sentence"])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
#             print("female_sentence: ", lines["sentence"])
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" #lines["sentid"]
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
#             print("female_sentence: ", lines["sentence"])
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" #lines["sentid"]
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1
#         if i >= 6:
#             break

In [198]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [199]:
# for i in range(len(list(noun_error.keys()))):
#     print(list(noun_error.keys())[i].lstrip("the").strip())

In [200]:
# random.choice()

In [201]:
fairness_errors = []
all_inputs = []
count = 0

In [202]:
# TODO:
#     * check for mispredictions
#     * save tested inputs, input pairs, inputs leading to fairness errors and prediction errors
#     * do for all test sets (a to f) i.e. other bias tests : name, race e.t.c

In [203]:
# TODO "the" vs. "The" in str.replace

In [204]:

for i in range(len(input_truple)):
#     print(input_truple[i])
#     print(input_truple[i][0])
#     print(input_truple[i][1])
#     print(occupation_list[i])
    if occupation_list[i] in wino_occupations:
#         for k in range(len(noun_error1))
#         for j in range(len(list(noun_error.keys()))):
#         for j in range(max([len(noun_error1), len(noun_error2)])):
        for j in range(len(noun_error1)) :
            count += 1
            input1, input2 = "", ""
#             input1 = input_truple[i][0].replace(occupation_list[i], list(noun_error.keys())[j].lstrip("the").strip())
#             input2 = input_truple[i][1].replace(occupation_list[i], list(noun_error.keys())[j].lstrip("the").strip())
            if ("the " + occupation_list[i]) in  input_truple[i][0]:
                input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))
            
        
            input1, input2 = "", ""
            if ("the " + occupation_list[i]) in  input_truple[i][1]:
                input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))   
    
    
            input1, input2 = "", ""
            if ("the " + occupation_list[i]) in  input_truple[i][2]:
                input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
            elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
            else:
                input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
            all_inputs.append((input1, input2))
            pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
            pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
#             print(pred1)
#             print(pred2)
            if (np.sign(pred1) != np.sign(pred2)):
#                 print(input1, pred1, np.sign(pred1))
#                 print(input2, pred2, np.sign(pred2))
                fairness_errors.append((input1, input2))   
                
            if len(all_inputs) >= 3000:
                break

    if len(all_inputs) >= 3000:
        break


In [205]:
len(all_inputs)

3000

In [206]:
len(fairness_errors)

257